This notebook is a "meta-notebook" in that it generates other notebooks to actually get run.

The substitution rules for template notebooks are:

* The template must start with "TEMPLATE", which will be removed in the output name.
* All outputs in the templates are removed
* Any text in the *first* code cell that is of the pattern "{T{SOMETHING}" will be replaced by whatever the tempate has for the key SOMETHING.
* Any text in the notebook filename that matches a key will be replaced

In [ ]:
import re
from pathlib import Path

import nbformat

from astroquery.mast import Observations

In [ ]:
PROPOSAL_ID = 2609

In [ ]:
def apply_to_templates(templates, template_mapping):
    outputs = {}
    for tname, template in templates.items():
        if not (tname.startswith('TEMPLATE') and tname.endswith('.ipynb')):
            raise ValueError('not a TEMPLATE notebook!')
            
        nb = nbformat.NotebookNode(template)  # copy to not overwrite the template

        cells = nb['cells']
        for cell in cells:
            cell['outputs'] = []
        src = cells[0]['source']
        for k, v in template_mapping.items():
            src = src.replace("{T{"+k+"}", v)
        cells[0]['source'] = src

        nbname = tname.removeprefix('TEMPLATE')
        for k, v in template_mapping.items(): 
            nbname = nbname.replace(k, v)
            
        outputs[nbname] = nb
    return outputs

In [ ]:
templates = {templatepath.name:nbformat.read(templatepath.open(), 4)
             for templatepath in list(Path('.').glob('TEMPLATE*.ipynb'))}

# Select the slits to run

In [ ]:
obses = Observations.query_criteria(proposal_id=PROPOSAL_ID)
specobses = obses[(obses['calib_level']==3)&(obses['dataproduct_type'] == 'spectrum')]
specobses

In [ ]:
target_name = 'NGC6791-NEWCATALOG-GAIAEDR3'
specnames = []
for row in specobses[specobses['target_name'] == target_name]:
    m = re.match('jw.*_(s.*?)_.*', row['obs_id'])
    if m is not None:
        specnames.append((target_name, m.group(1)))

# Generate the notebooks

In [ ]:
nbpaths_to_exec = []
for targetname, slitname in specnames:
    outputs = apply_to_templates(templates,
                                   {'DLPATHNAME': 'mastDownload/flat/',
                                    'REDUXBASENAME': 'miscdata/jwst_outputs/',
                                    'PROPOSALID': str(PROPOSAL_ID),
                                    'SLITNAME': slitname,
                                    'TARGETNAME': targetname,
                                   })
    for k,v in outputs.items():
        outfn = Path('miscdata/jwst_outputs') / k
        print('writing to', outfn)
        nbformat.write(v, outfn)
        nbpaths_to_exec.append(outfn)

# Commands to execute the notebooks

In [ ]:
for nbp in nbpaths_to_exec:
    print(f'jupyter nbconvert --to notebook --execute {nbp}')

Or for split into pipeline and non-pipeline ones:

In [ ]:
for nbp in nbpaths_to_exec:
    if 'pipeline' in str(nbp):
        print(f'jupyter nbconvert --to notebook --execute {nbp}')

In [ ]:
for nbp in nbpaths_to_exec:
    if 'pipeline' not in str(nbp):
        print(f'jupyter nbconvert --to notebook --execute {nbp}')